In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

## T1

In [2]:
def gaussian_eq(x, mean, var):
    return np.diagonal(np.exp(-0.5 * (x - mean).dot(np.linalg.inv(var)).dot((x - mean).T))
                       / np.sqrt(2*np.pi * np.linalg.det(var)))

def soft_em(data, k, means, iters):
    # Initialize step
    m = np.array([1/k] * k, dtype=float)
    var = np.array([np.identity(2) for i in range(k)], dtype=float)
    
    for it in range(iters):
        # Expectation
        w = np.zeros((data.shape[0], k))
        for j in range(k):
            w[:, j] = gaussian_eq(data, means[j], var[j]) * m[j]
        w = np.divide(w, np.sum(w, axis=1)[:, None])
        
        # Maximization
        for j in range(k):
            m[j] = w[:, j].sum() / data.shape[0]
            var_j = np.zeros((2, 2))
            for i in range(data.shape[0]):
                var_j += w[i, j] * (data[i] - mean[j]).T.dot(data[i] - mean[j])
            var_j /= np.sum(w[:, j])
            var_j[0, 1] = var_j[1, 0] = 0
            var[j] = var_j
            mean[j] = np.sum(data * w[:, j].reshape(-1, 1)) / np.sum(w[:, j])
            
        # Display Result
        print("Iteration: ", it)
        print("w: ", w)
        print("m: ", m)
        print("Cov: ", var)
        print("\n", "="*50, "\n")
    return w

In [3]:
data_x = np.array([1, 3, 2, 8, 6, 7, -3, -2, -7])
data_y = np.array([2, 3, 2, 8, 6, 7, -3, -4, -7])
data = np.array([data_x, data_y]).T
data

array([[ 1,  2],
       [ 3,  3],
       [ 2,  2],
       [ 8,  8],
       [ 6,  6],
       [ 7,  7],
       [-3, -3],
       [-2, -4],
       [-7, -7]])

In [4]:
mean_x = np.array([3, 2, -3])
mean_y = np.array([3, 2, -3])
mean = np.array([mean_x, mean_y]).T
mean

array([[ 3,  3],
       [ 2,  2],
       [-3, -3]])

In [5]:
w = soft_em(data, 3, mean, 3)

Iteration:  0
w:  [[  1.19202922e-01   8.80797076e-01   1.81545808e-09]
 [  7.31058579e-01   2.68941421e-01   1.69570706e-16]
 [  2.68941421e-01   7.31058579e-01   1.01529005e-11]
 [  9.99983299e-01   1.67014218e-05   2.03105874e-42]
 [  9.99088949e-01   9.11051194e-04   5.37528453e-32]
 [  9.99876605e-01   1.23394576e-04   3.30529272e-37]
 [  2.31952283e-16   1.38879439e-11   1.00000000e+00]
 [  2.31952283e-16   1.38879439e-11   1.00000000e+00]
 [  3.30570063e-37   5.90009054e-29   1.00000000e+00]]
m:  [ 0.45757242  0.20909425  0.33333333]
Cov:  [[[ 24.55293548   0.        ]
  [  0.          24.55293548]]

 [[  0.77328542   0.        ]
  [  0.           0.77328542]]

 [[ 11.33333335   0.        ]
  [  0.          11.33333335]]]


Iteration:  1
w:  [[  4.19395510e-02   9.57161798e-01   8.98651358e-04]
 [  5.05955731e-03   9.94937945e-01   2.49794320e-06]
 [  9.18771589e-03   9.90754462e-01   5.78225470e-05]
 [  1.00000000e+00   1.90667375e-13   3.52704929e-10]
 [  9.99645793e-01   3.54

In [6]:
clusters = [[] for i in range(w.shape[1])]
for i, row in enumerate(w):
    clusters[np.argmax(row)].append(data[i])
for i, cluster in enumerate(clusters):
    cluster = np.array(cluster)
    plt.scatter(cluster[:, 0], cluster[:, 1], color=["red", "green", "blue"][i])
plt.show()